# Extracción de datos a través de *Beautiful Soap 4*

## 1.- Obtención de datos general de los datos.

En un primer paso vamos a hacer llamadas a través de la librería de BeautifulSoap4 para obtener datos generales desde la primera pagina.

Como siempre lo primero es importar las librerías 

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [23]:
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online'

### Conectamos la pagina web para acceder a ella y *arañar* los datos de su página web.

In [2]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
response = requests.get(url)

In [3]:
html = response.content
soup = bs(html, "lxml")

#### a.- Vamos a obtener información de los datos existente en la URL.

In [ ]:
all_h3 = soup.find_all("h3")
all_h3

#### b.- Extraemos los nombres desde esta URL.

In [4]:
titulos=soup.find_all("h3")
name=[]
for titulo in titulos[1:]:
    nombre=titulo.get_text(strip=True).split(',')[0]
    name.append(nombre)
print(len(name))
print(name)

24
['LIZO 2', 'FOXTAIL', 'TOBOGANE HOT RABBIT', 'Vibrador Líquido con sabor Desliz! VIBRAGEL 30ml', 'MENEO sube y baja', 'TOBOGANE', 'CRISTALINO XL', 'FRESH GIRL', 'BALLENATO', 'Bacanal Gel Anal monodosis', 'FLOGGY - Flogger BDSM de piel vegana', 'CAMILLE', 'SAZZIA', 'AVATAR FIRST LOVER', 'AVENTURE -  Vibrador con imán y mando a distancia', 'REGGIA', 'MAGIC CUP', 'BUTTERFLY', 'EXPLORE PERFECT', 'Vibrador Líquido con sabor Desliz! VIBRAGEL 10ml', 'EISSELY', 'TRIS-TRAS', 'TOK anal 10', 'TROMPI']


#### c.- Con este codigo extraigo la descripción del producto.

In [5]:
titulos=soup.find_all("h3")
desc=[]

for titulo in titulos[1:]:
    description=titulo.get_text(strip=True).split(', ')[1:]
    if description==[]:
        description=["No hay datos"]
    desc.append(description)
print(len(desc))
print(desc)

24
[['Dildo de suave silicona en 3 tamaños'], ['plug anal cola de zorro de 35cm'], ['el superventas de amantis ¡mejorado!'], ['hormigueo...'], ['placer realista con control remoto'], ['el vibrador doble más vendido'], ['gran dildo transparente de 22cm'], ['6 Kilos y 40cm de piel real disfrutable'], ['tu vibrador a distancia con aleta móvil y sumergible...'], ['Pack de 10 uds'], ['No hay datos'], ['body camiseta floral con cuello halter de amantis'], ['masturbador hiperrealista'], ['amante realista de suave silicona'], ['No hay datos'], ['masturbador masculino doble'], ['Masturbarse mola un huevo'], ['Bodysocking de fina malla con detalle floral'], ['posiblemente el dildo realista perfecto'], ['hormigueo...'], ['medias abiertas para recorrer'], ['anilla con vibrador para doble penetración TRINITY...'], ['Vibrador anal con 10 modos de vibración'], ['vibrador sumergible ideal para Punto-G']]


Vemos que en algunos casos no hay información. 

En otros casos la separación de la descripción, no se ha separado.

Por este motivo, haremos la extracción unicamente de todo y posteriormente lo trataremos con la librería de *pandas*.

#### d.- Con esto extraigo los links de los productos.

In [6]:
productos = soup.find_all(class_='caption')
lista_URLs = []
for producto in productos[8:]:
    URL_producto = producto.find('a')['href']
    lista_URLs.append(URL_producto)

len(lista_URLs)

24

Esta información es importante ya que nos permitirá obtener las direcciones de cada producto para hacer posteriormente la información desde cada producto.

#### e.- Obtenemos la información de precio rebajado de cada producto.

In [7]:
all_price=soup.find_all("span",class_="productSpecialPrice")
price=[]

for precio in all_price:
    item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
    price.append(item_price)
len(price)


23

#### Este es el codigo completo para extraer la información de cada página.

In [8]:
titulos=soup.find_all("h3")
name=[]
desc=[]
price=[]
lista_URLs = []

for titulo in titulos:
    nombre=titulo.get_text(strip=True).split(',')[0]
    name.append(nombre)
    description=titulo.get_text(strip=True).split(', ')[1:]
    if description==[]:
        description=["No hay datos"]
    desc.append(description)

all_price=soup.find_all("span",class_="productSpecialPrice")

for precio in all_price:
    item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
    price.append(item_price)

productos = soup.find_all(class_='caption')
for producto in productos[8:]:
    URL_producto = producto.find('a')['href']
    lista_URLs.append(URL_producto)

df_productos = pd.DataFrame({"Name": name,"Description": desc,"Price":price,"link":lista_URLs})
df_productos

ValueError: All arrays must be of the same length

In [9]:
name[0]

'Top ventas en amantis'

In [10]:
lista_URLs[0]

'https://www.amantis.net/lizo-2-dildo-suave-silicona-3-tamanos/'

## Vamos a sacar la información de todas las páginas posibles.

Para esto vamos a obtener las URLs, como en el punto d. 

Para extraer los links de los productos y ver si podemos sacar la información de su página concreta.

In [11]:
lista_URLs[0]

'https://www.amantis.net/lizo-2-dildo-suave-silicona-3-tamanos/'

In [12]:
# Hacemos un nuevo request para el primer libro: 
r = requests.get(lista_URLs[0])

# Creamos una sopa específica con la info de cada libro
soup_producto = bs(r.text, "lxml")

In [ ]:
soup_producto

In [13]:
name = soup_producto.find('h1').text
print(name)

LIZO 2, Dildo de suave silicona en 3 tamaños


#### a.- Vamos a entrar en una página general para extraer información de ella.

Principalmente vamos a ver como podemos, con un bucle, sacar todos los datos de cada producto.

In [ ]:
page=3
url = "https://www.amantis.net/productos-amantis/"              # lista productos
URL = url+'page' + str(page)+'/'
response = requests.get(URL)
titulos=soup.find_all("h3")
titulos

Este es el bucle para obtener información anterior de las 5 primeras paginas.

In [ ]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
response = requests.get(url)


pages= np.arange(2,5)                                   # La primera pagina tiene una serie de datos que no debemos de recoger.
name=[]
desc=[]
price=[]
lista_URLs = []

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)
            print(nombre)

        all_price=soup.find_all("span",class_="productSpecialPrice")

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)


        productos = soup.find_all(class_='caption')

        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
            print(URL_producto)
            
            
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)
            # print(nombre)

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)

        productos = soup.find_all(class_='caption')

        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
            # print(URL_producto)

print("Nombres: ",len(name))
print("Descripción: ",len(desc))
print("URL: ",len(lista_URLs))
print("Precios: ",len(price))

Pagina 2
Pagina 3
Pagina 4
Nombres:  72
Descripción:  72
URL:  72
Precios:  69


Dado que vemos que hay incongruencias en la longitud del tamaño de cada lista, vamos a realizar un Dataframe unicamente de los datos con la misma longitud.

Esta problemática la solucionaremos más adelante.

In [ ]:
df_productos = pd.DataFrame({"Name": name,"Description": desc,"link":lista_URLs})
df_productos.head()

,Name,Description,link
0,BISOU,[besos por ondas de succión con la mejor vibra...,https://www.amantis.net/bisou-besos-ondas-succ...
1,POWER UP METER - Bomba de succión con manómetro,[No hay datos],https://www.amantis.net/power-up-meter-bomba-s...
2,TOK2 bala vibradora con mando,[más potente y recargable],https://www.amantis.net/tok2-bala-vibradora-ma...
3,MAGIC CUP,[Masturbarse mola un huevo],https://www.amantis.net/magic-cup-6-masturbado...
4,TRIPLE RABBIT,[Conejito vibrador con bolas anales],https://www.amantis.net/triple-rabbit-conejito...


#### b.- Verifico que he conseguido las URLs de cada producto, esto es importante para obtener la información de cada producto.

Extraer la información de producto, descripción, enlace y precio tomando los datos desde las URLs de cada producto.

Queda pendiente extraer información de los ratings y los comentarios para establecer un estudio

In [14]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
response = requests.get(url)


pages= np.arange(1, 25)
name=[]
desc=[]
price=[]
lista_URLs = []

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos[1:]:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)

        all_price=soup.find_all("span",class_="productSpecialPrice")

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)


        productos = soup.find_all(class_='caption')

        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
            
            
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)

        productos = soup.find_all(class_='caption')

        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)




# df_productos = pd.DataFrame({"Name": name,"Description": desc,"Price":price,"link":lista_URLs})
# df_productos.head()


Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20
Pagina 21
Pagina 22
Pagina 23
Pagina 24


Verificamos que cantidad de datos tenemos al recorrer todos las paginas con productos.

In [16]:
print("Nombres:" ,len(name))
print("descrip:" ,len(desc))
print("precio:" ,len(price))                    #  Se ve que hay un desajuste en el precio al extraer la información de 1 pagina
print("URL:" ,len(lista_URLs))

Nombres: 576
descrip: 576
precio: 552
URL: 576


In [17]:
print("Nombres:\n" ,name[:5])
print("descrip:\n" ,desc[:5])
print("precio:\n" ,price[:5])
print("URL:\n" ,lista_URLs[:5])

Nombres:
 ['LIZO 2', 'FOXTAIL', 'TOBOGANE HOT RABBIT', 'Vibrador Líquido con sabor Desliz! VIBRAGEL 30ml', 'MENEO sube y baja']
descrip:
 [['Dildo de suave silicona en 3 tamaños'], ['plug anal cola de zorro de 35cm'], ['el superventas de amantis ¡mejorado!'], ['hormigueo...'], ['placer realista con control remoto']]
precio:
 ['17.99', '9.99', '39.99', '9.99', '44.99']
URL:
 ['https://www.amantis.net/lizo-2-dildo-suave-silicona-3-tamanos/', 'https://www.amantis.net/foxtail-plug-anal-cola-zorro/', 'https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/', 'https://www.amantis.net/desliz-vibragel-liquido-vibrador-30ml/', 'https://www.amantis.net/meneo-sube-baja-realista-control-remoto/']


#### c.- Extrayendo la información de los comentarios de cada producto.

Vamos a entrar en un producto para obtener la información de los comentarios que hay:
- Usuario
- Fecha
- Comentario
- Rating 

In [ ]:
prueba=lista_URLs[1]
response = requests.get(prueba)
soup_prueba = bs(response.text, 'lxml')

titulo=soup_prueba.get_text(strip=True).split(',')[0]
print(titulo)

BALLENATO


**Fechas**

Estos datos son en este momento, *string*.

In [ ]:
rating=[]
all_ratings = soup_prueba.find_all("span", class_="date")  
sep_1=(' ')
sep_2=(', ')
# pattern = re.compile(sep)
for ratings in all_ratings:
    rating_coment_1=ratings.get_text(strip=True).split(sep_2)[1]
    rating_coment_2=ratings.get_text(strip=True).split(sep_2)[0]
    rating_coment_3=rating_coment_2.split(sep_1)[1:]
    date_1=rating_coment_3[0]+"/"+rating_coment_3[1]
    date_2=date_1+"/"+rating_coment_1                                       # Estamos pendientes de convertir a fechas, teniendo en cuenta
    # date_object = datetime.strptime(date_2,'%d%m%Y')                        # que esta en español
    # print(type(date_1))
    # print(date_1)
    rating.append(date_2)
    # print(type(date_2))
    # print(date_2)

print(len(rating))
rating

20


['18/febrero/2023',
 '01/febrero/2023',
 '18/noviembre/2022',
 '06/septiembre/2022',
 '13/julio/2022',
 '17/junio/2022',
 '05/noviembre/2021',
 '06/julio/2021',
 '24/mayo/2021',
 '16/mayo/2021',
 '30/abril/2021',
 '16/abril/2021',
 '01/marzo/2021',
 '09/febrero/2021',
 '13/enero/2021',
 '26/noviembre/2020',
 '04/noviembre/2020',
 '25/septiembre/2020',
 '21/septiembre/2020',
 '14/septiembre/2020']

**Usuarios**

In [ ]:
user_comments=[]
all_user_comments = soup_prueba.find_all("span", class_="name-user")  

for user_comment in all_user_comments:
    user_comments.append(user_comment.get_text(strip=True))

print(len(user_comments))
user_comments

20


['Almudena',
 'Tomabel',
 'andrea',
 'Carlos',
 'maria',
 'Mar',
 'Sasha',
 'Andrea',
 'irene',
 'Jose',
 'Ines',
 'Rafael',
 'andrea',
 'Alejandro',
 'Elisabet',
 'Marta',
 'Ana',
 'Joana',
 'Ricardo',
 'fernando']

**Comentarios**

In [ ]:
comment=[]
all_comments = soup_prueba.find_all("p")
for formats in all_comments[-len(rating):]:                     # Creo que me he cargado 'rating'
    comment.append(formats.get_text(strip=True))

print(comment[1])
len(comment)


Fue mi regalo de cumpleaños...el mejor regalo de mi vida!!!! Muy fácil de poner y muy cómodo.ideal para penetración doble vaginal y dobletes.lo recomiendo 100x100!!!!!


20

Hasta aquí los datos que estamos obteniendo se pasan a una lista por atributo.

### d.- Este es el código bueno para iterar todos los productos extraer la información siguiente:

- Nombre
- Descripción
- Precio sin rebaja (regular_price)
- Precio rebajado (new_price)
- Información de cada producto
- Lista de usuarios que han comentado
- Lista de comentarios
- Lista de Fecha de comentarios
- Lista de Ratings.


In [14]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
# pages= np.arange(1,5)
pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []
name=[]
subname=[]
regular_prices=[]
new_price=[]
# lista_URLs = []
info=[]
charac=[]
user_comments=[]
comment=[]
date=[]
ratings=[]

''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)

'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]
    

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True).split(',')[0]
        name.append(nombre)
        sub_title=titulo.get_text(strip=True).split(', ')[1:]
        if sub_title==[]:
            sub_title=["No hay datos"]
        subname.append(sub_title)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:5]
    info.append(information)
    characteristic=description.get_text().split('\n')[5:-3]
    charac.append(characteristic)
    '''Vamos a obtener los datos de los comentarios de los usuarios'''

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    user_comments.append(user_comments_product)

    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)
    date.append(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    comment.append(comments_product)

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)
    ratings.append(rating)


for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]


# print('-'*20)
# print(name)
print('-'*20)
print(len(name))
# print('-'*20)
# print(subname)
print('-'*20)
print(len(subname))
print('-'*20)
# print(regular_prices)
# print('-'*20)
print(len(regular_prices))
# print('-'*20)
# print(new_price)
print('-'*20)
print(len(new_price))
print('-'*20)
print(len(lista_URLs))
# print('-'*20)
# print(lista_URLs)
print('-'*20)
print(len(info))
# print('-'*20)
# # print(charac)
print('-'*20)
print(len(charac))
# print('-'*20)
# # print(user_comments)
print('-'*20)
print(len(user_comments))
# print('-'*20)
# # print(comment)
print('-'*20)
print(len(comment))
# # print('-'*20)
# # print(date)
print('-'*20)
print(len(date))
print('-'*20)
print(len(ratings))

Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20
Pagina 21
Pagina 22
Pagina 23
Pagina 24
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576


## Visualizamos los datos para ver qué se obtiene y lo guardamos en un fichero .csv.

La intención es depurar el codigo para un mejor manejo de los datos.

In [15]:
dataframe= pd.DataFrame({"Name": name,"Subname": subname,"Description": info, "Characteristics": charac,"Price":regular_price,"Reduced Price":new_price,
                         "date":date,"User": user_comments,"Ratings": ratings,"Comment": comment})
dataframe

,Name,Subname,Description,Characteristics,Price,Reduced Price,date,User,Ratings,Comment
0,TOBOGANE HOT RABBIT,[el superventas de amantis ¡mejorado!],[Vuelve nuestro vibrador de doble estimulación...,"[, , Medidas: 19cm (11cm insertables) y 3,3cm/...",21.99,39.99,"[martes 22 noviembre, 2022, jueves 07 julio, 2...","[Rossi, Marina, Jennifer, Noa, Karen, Lorena, ...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","[Mi primera compra. Me encantó la textura, los..."
1,LIZO 2,[Dildo de suave silicona en 3 tamaños],"[En un azulejo de la cocina, en una puerta, en...",[Tanto para tus momentos de onanismo como para...,21.99,17.99,"[domingo 12 marzo, 2023, martes 10 enero, 2023...","[Iria, Barney, Sara, Aida, Lucas, antonio, Jes...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",[Al no tener tope viene perfecto para usar con...
2,FOXTAIL,[plug anal cola de zorro de 35cm],[Deja volar la imaginación y saca tu lado más ...,"[, , Tamaño plug S: 6.6cm Max Dia.: 2.8cm, Col...",21.99,9.99,"[jueves 26 enero, 2023, jueves 05 enero, 2023,...","[Jonatan, Irene, Ainara, Ignacio, andrea, Alic...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4]","[Un buen plug, muy suave, aunque con el lubric..."
3,TOBOGANE,[el vibrador doble más vendido],"[Por favor, desabróchense los cinturones de se...","[, La última atracción exclusiva de amantis es...",21.99,36.99,"[lunes 06 marzo, 2023, sábado 04 marzo, 2023, ...","[Teresa, Alicia, María, Sara, Raquel, Sara, Da...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",[Fue el primer vibrador que compré. Me lo reco...
4,MENEO sube y baja,[placer realista con control remoto],"[Si te gusta que te metan un buen meneo, hazte...","[Ya lo ves, Meneo’s Cock puede ir contigo a do...",21.99,44.99,"[miércoles 14 diciembre, 2022, miércoles 09 no...","[Francisco, Maria, Jose Javier, Carlos, Pedro,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",[Quería saber cuantos cm tiene la longitud que...
...,...,...,...,...,...,...,...,...,...,...
571,JUSTISSE NECK - collar + bloqueadores de muñecas,[No hay datos],"[Muy a menudo, las parejas se quejan de que la...","[\r, Con este kit JUSTISSE NECK solo tienes un...",21.99,23.99,"[domingo 08 noviembre, 2020]",[Alejandro],[5],[Es un poco engorroso a la hora de ponerlo sob...
572,MS-VULVAN. Estimulador Total de Vulva (Cabezal...,[No hay datos],[La orgásmica colección de accesorios para aco...,"[\r, Realizados en un agradable TPR liso y sin...",21.99,8.99,"[jueves 02 agosto, 2018, martes 17 julio, 2018...","[Inma, yulema, Cristina, Veturian, Pilar, Iván...","[5, 5, 3, 5, 5, 4, 3]","[Es uno de mis cabezales estrella, es muy como..."
573,AVATAR SIMON PRIDE,[Orgulloso dildo de silicona],[¿Quién dijo que en el mundo de los dildos no ...,"[, Juguete de silicona 100%, Dimensiones: 20 c...",21.99,34.99,"[martes 06 agosto, 2019]",[Javier],[4],[A mi chica no le gustaba la típica apariencia...
574,TSUNAMI FORMAS,[Fundas de silicona para TSUNAMI POWER],[¿Has echado un vistazo a nuestro productazo l...,"[, , Un dildo con anilla, porque hay veces en ...",21.99,14.99,"[miércoles 11 noviembre, 2020, miércoles 01 ju...","[Christian, Carmen, Meritxell, Esther, L04]","[5, 5, 4, 5, 5]",[Imprescindibles para disfrutar del Tsunami co...


In [24]:
file=folder+'\\Data\\scrapped_data.csv'
file

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online\\Data\\scrapped_data.csv'

In [17]:
dataframe.to_csv(file,header=True,index=False)           # Tengo que generar el path correcto


## Solucionando inconvenientes de los datos obtenidos.

- *Subname* puede generar problemas por lo eliminaremos del codigo. Lo generaremos después a partir de *Name*.
- *Characteristics* puede generar problemas, ya que serían datos de dimensiones, duración, etc. Lo generaremos después a partir de *Description*.
- Hay que pasar las listas a registros individuales.

In [20]:
lista_URLs[0]

'https://www.amantis.net/lizo-2-dildo-suave-silicona-3-tamanos/'

In [ ]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
# pages= np.arange(1,5)
pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []

''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Leyendo paginas")
        # print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
#        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
print("Terminando lectura.\nRecabando información.")

list_name=[]
list_regular_prices=[]
list_new_price=[]
list_info=[]
user_comments=[]
comment=[]
date=[]
ratings=[]
datas=[]
diccionario_URL={}



'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]
    data=[]
    name=[]
    regular_prices=[]
    new_price=[]
    info=[]

    diccionario_comments={}

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True)
        name.append(nombre)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:]
    documentation = ''.join(information)
    info.append(documentation)


    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    for i, regular_price in enumerate(regular_prices):
        if regular_price is None:
            regular_prices[i] = new_price[i]


    '''Vamos a obtener los datos de los comentarios de los usuarios'''
    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        date_comments_product.append(dates_text)
    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = str(len(heart_rating))
        rating.append(num_hearts)
    
    # diccionario={}
    user_comments.append(user_comments_product)
    date.append(date_comments_product)
    comment.append(comments_product)
    ratings.append(rating)
    
    # datos = list(zip(nombre,user_comments_product,comments_product, date_comments_product, rating))
    # for dato in datos:
    #     nombre=dato[0]
    #     comentarios=dato[1:]
    #     if nombre in diccionario_comments:
    #         diccionario_comments[nombre].append(comentarios)
    #     else:
    #         diccionario_comments[nombre] = [comentarios]
    # # diccionario_URL[name] = name
    # diccionario_URL[URL] = diccionario_comments
    
    
# diccionario_URL


dataframe= pd.DataFrame({"Name": name,"Description": info,"Price":regular_price,"Reduced Price":new_price,
                         "date":date,"User": user_comments,"Ratings": ratings,"Comment": comment
                        #  ,"diccionario":diccionario
                        })
dataframe

### Realizando las sustituciones pertinentes

In [20]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
# pages= np.arange(1,5)
pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []
name=[]
regular_prices=[]
new_price=[]
info=[]
user_comments=[]
comment=[]
date=[]
ratings=[]

''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[8:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)

'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]
    

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True)
        name.append(nombre)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:]
    documentation = ''.join(information)
    info.append(documentation)


    '''Vamos a obtener los datos de los comentarios de los usuarios'''

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    user_comments.append(user_comments_product)

    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)
    date.append(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    comment.append(comments_product)

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)
    ratings.append(rating)


for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]


# print('-'*20)
# print(name)
print('-'*20)
print(len(name))
# print('-'*20)
# print(subname)
# print('-'*20)
# print(len(subname))
print('-'*20)
# print(regular_prices)
# print('-'*20)
print(len(regular_prices))
# print('-'*20)
# print(new_price)
print('-'*20)
print(len(new_price))
print('-'*20)
print(len(lista_URLs))
# print('-'*20)
# print(lista_URLs)
print('-'*20)
print(len(info))
# print('-'*20)
# # print(charac)
# print('-'*20)
# print(len(charac))
# print('-'*20)
# # print(user_comments)
print('-'*20)
print(len(user_comments))
# print('-'*20)
# # print(comment)
print('-'*20)
print(len(comment))
# # print('-'*20)
# # print(date)
print('-'*20)
print(len(date))
print('-'*20)
print(len(ratings))

Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20
Pagina 21
Pagina 22
Pagina 23
Pagina 24
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576


In [12]:
print(len(info))

1


In [ ]:
diccionario_URL

In [21]:
dataframe= pd.DataFrame({"Name": name,"Description": info,"Price":regular_price,"Reduced Price":new_price,
                         "date":date,"User": user_comments,"Ratings": ratings,
                         "Comment": comment})
dataframe

,Name,Description,Price,Reduced Price,date,User,Ratings,Comment
0,"TOBOGANE HOT RABBIT, el superventas de amantis...",Vuelve nuestro vibrador de doble estimulación ...,21.99,39.99,"[martes 22 noviembre, 2022, jueves 07 julio, 2...","[Rossi, Marina, Jennifer, Noa, Karen, Lorena, ...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","[Mi primera compra. Me encantó la textura, los..."
1,"LIZO 2, Dildo de suave silicona en 3 tamaños","En un azulejo de la cocina, en una puerta, en ...",21.99,17.99,"[domingo 12 marzo, 2023, martes 10 enero, 2023...","[Iria, Barney, Sara, Aida, Lucas, antonio, Jes...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",[Al no tener tope viene perfecto para usar con...
2,"FOXTAIL, plug anal cola de zorro de 35cm",Deja volar la imaginación y saca tu lado más s...,21.99,9.99,"[jueves 26 enero, 2023, jueves 05 enero, 2023,...","[Jonatan, Irene, Ainara, Ignacio, andrea, Alic...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4]","[Un buen plug, muy suave, aunque con el lubric..."
3,"TOBOGANE, el vibrador doble más vendido","Por favor, desabróchense los cinturones de seg...",21.99,36.99,"[lunes 06 marzo, 2023, sábado 04 marzo, 2023, ...","[Teresa, Alicia, María, Sara, Raquel, Sara, Da...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",[Fue el primer vibrador que compré. Me lo reco...
4,"MENEO sube y baja, placer realista con control...","Si te gusta que te metan un buen meneo, hazte ...",21.99,44.99,"[miércoles 14 diciembre, 2022, miércoles 09 no...","[Francisco, Maria, Jose Javier, Carlos, Pedro,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",[Quería saber cuantos cm tiene la longitud que...
...,...,...,...,...,...,...,...,...
571,JUSTISSE NECK - collar + bloqueadores de muñecas,"Muy a menudo, las parejas se quejan de que la ...",21.99,23.99,"[domingo 08 noviembre, 2020]",[Alejandro],[5],[Es un poco engorroso a la hora de ponerlo sob...
572,MS-VULVAN. Estimulador Total de Vulva (Cabezal...,La orgásmica colección de accesorios para acop...,21.99,8.99,"[jueves 02 agosto, 2018, martes 17 julio, 2018...","[Inma, yulema, Cristina, Veturian, Pilar, Iván...","[5, 5, 3, 5, 5, 4, 3]","[Es uno de mis cabezales estrella, es muy como..."
573,"AVATAR SIMON PRIDE, Orgulloso dildo de silicona",¿Quién dijo que en el mundo de los dildos no h...,21.99,34.99,"[martes 06 agosto, 2019]",[Javier],[4],[A mi chica no le gustaba la típica apariencia...
574,"TSUNAMI FORMAS, Fundas de silicona para TSUNAM...",¿Has echado un vistazo a nuestro productazo la...,21.99,14.99,"[miércoles 11 noviembre, 2020, miércoles 01 ju...","[Christian, Carmen, Meritxell, Esther, L04]","[5, 5, 4, 5, 5]",[Imprescindibles para disfrutar del Tsunami co...


In [25]:
dataframe.to_csv(file,header=True,index=False)           # Tengo que generar el path correcto

In [42]:
prueba=dataframe[['Name','date','User','Ratings','Comment']]
prueba

,Name,date,User,Ratings,Comment
0,"TOBOGANE HOT RABBIT, el superventas de amantis...","[martes 22 noviembre, 2022, jueves 07 julio, 2...","[Rossi, Marina, Jennifer, Noa, Karen, Lorena, ...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","[Mi primera compra. Me encantó la textura, los..."
1,"BALLENATO, tu vibrador a distancia con aleta m...","[sábado 18 febrero, 2023, miércoles 01 febrero...","[Almudena, Tomabel, andrea, Carlos, maria, Mar...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[Sigo temblando con este juguete, menudas vibr..."
2,"FRESH GIRL, 6 Kilos y 40cm de piel real disfru...","[sábado 31 diciembre, 2022, viernes 21 octubre...","[Adrian, Daniel, victor, Guillermo, Jesús, Dav...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[Este juguete es bastante bueno, muy muy place..."
3,"TOBOGANE, el vibrador doble más vendido","[viernes 24 febrero, 2023, miércoles 25 enero,...","[María, Sara, Raquel, Sara, Daniel, Pilar, Dav...","[3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, ...","[El producto está muy bien, muy suave, tamaño ..."
4,"MENEO sube y baja, placer realista con control...","[miércoles 14 diciembre, 2022, miércoles 09 no...","[Francisco, Maria, Jose Javier, Carlos, Pedro,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",[Quería saber cuantos cm tiene la longitud que...
...,...,...,...,...,...
571,Jaula de castidad masculina de acero con BARROTES,"[jueves 26 agosto, 2021, domingo 13 diciembre,...","[Diego, Ricardo, Joe, Ignasi, Miguel Angel, Sara]","[4, 5, 3, 5, 5, 5]","[Hola, la compró mi chica por sorpresa después..."
572,"COOL CUP, Cápsula para masturbación reutilizab...","[sábado 09 abril, 2022, jueves 10 marzo, 2022,...","[Ernesto, Miguel Ángel, Miguel Ángel, Juan Fra...","[5, 5, 5, 1, 5, 5, 5, 4, 5, 4, 5, 4, 1, 4, 3, ...","[Tacto muy suave, me gusta porque coje todo el..."
573,SAUVAGE CAT- Plug rosa con cola blanca o negra,"[miércoles 30 noviembre, 2022, lunes 17 enero,...","[roberto, Miguel, David, Maria Elena, Aroa, Ju...","[5, 4, 5, 4, 5, 5, 4, 4, 5, 5, 5, 5, 5, 4, 5, ...",[Suave y sexy colita de gato. El plug en color...
574,"Xtreme, dos huevitos vibradores a compartir......","[miércoles 21 abril, 2021, sábado 13 junio, 20...","[Ana, ruben, Marta, Joan]","[4, 5, 3, 5]",[Luces y sombras en este juguetito:rnrnComo co...


In [ ]:
prueba

,Name,date,User,Ratings,Comment
0,"TOBOGANE HOT RABBIT, el superventas de amantis...","[martes 22 noviembre, 2022, jueves 07 julio, 2...","[Rossi, Marina, Jennifer, Noa, Karen, Lorena, ...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","[Mi primera compra. Me encantó la textura, los..."
1,"BALLENATO, tu vibrador a distancia con aleta m...","[sábado 18 febrero, 2023, miércoles 01 febrero...","[Almudena, Tomabel, andrea, Carlos, maria, Mar...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[Sigo temblando con este juguete, menudas vibr..."
2,"FRESH GIRL, 6 Kilos y 40cm de piel real disfru...","[sábado 31 diciembre, 2022, viernes 21 octubre...","[Adrian, Daniel, victor, Guillermo, Jesús, Dav...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[Este juguete es bastante bueno, muy muy place..."
3,"TOBOGANE, el vibrador doble más vendido","[viernes 24 febrero, 2023, miércoles 25 enero,...","[María, Sara, Raquel, Sara, Daniel, Pilar, Dav...","[3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, ...","[El producto está muy bien, muy suave, tamaño ..."
4,"MENEO sube y baja, placer realista con control...","[miércoles 14 diciembre, 2022, miércoles 09 no...","[Francisco, Maria, Jose Javier, Carlos, Pedro,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",[Quería saber cuantos cm tiene la longitud que...
...,...,...,...,...,...
571,Jaula de castidad masculina de acero con BARROTES,"[jueves 26 agosto, 2021, domingo 13 diciembre,...","[Diego, Ricardo, Joe, Ignasi, Miguel Angel, Sara]","[4, 5, 3, 5, 5, 5]","[Hola, la compró mi chica por sorpresa después..."
572,"COOL CUP, Cápsula para masturbación reutilizab...","[sábado 09 abril, 2022, jueves 10 marzo, 2022,...","[Ernesto, Miguel Ángel, Miguel Ángel, Juan Fra...","[5, 5, 5, 1, 5, 5, 5, 4, 5, 4, 5, 4, 1, 4, 3, ...","[Tacto muy suave, me gusta porque coje todo el..."
573,SAUVAGE CAT- Plug rosa con cola blanca o negra,"[miércoles 30 noviembre, 2022, lunes 17 enero,...","[roberto, Miguel, David, Maria Elena, Aroa, Ju...","[5, 4, 5, 4, 5, 5, 4, 4, 5, 5, 5, 5, 5, 4, 5, ...",[Suave y sexy colita de gato. El plug en color...
574,"Xtreme, dos huevitos vibradores a compartir......","[miércoles 21 abril, 2021, sábado 13 junio, 20...","[Ana, ruben, Marta, Joan]","[4, 5, 3, 5]",[Luces y sombras en este juguetito:rnrnComo co...


In [21]:
lista_URLs[0]

'https://www.amantis.net/lizo-2-dildo-suave-silicona-3-tamanos/'

In [127]:
dataframe.explode('datos')

,Name,datos
0,https://www.amantis.net/lizo-2-dildo-suave-sil...,Iria
0,https://www.amantis.net/lizo-2-dildo-suave-sil...,Al no tener tope viene perfecto para usar con ...
0,https://www.amantis.net/lizo-2-dildo-suave-sil...,"domingo 12 marzo, 2023"
0,https://www.amantis.net/lizo-2-dildo-suave-sil...,5
1,https://www.amantis.net/lizo-2-dildo-suave-sil...,Barney
...,...,...
53,https://www.amantis.net/lizo-2-dildo-suave-sil...,5
54,https://www.amantis.net/lizo-2-dildo-suave-sil...,Ariadna
54,https://www.amantis.net/lizo-2-dildo-suave-sil...,"Tacto muy bueno y agradable, se fija muy bien ..."
54,https://www.amantis.net/lizo-2-dildo-suave-sil...,"jueves 29 octubre, 2020"


In [130]:
dataframe['datos'][0]

('Iria',
 'Al no tener tope viene perfecto para usar con arnés o con otros juegos que requieran esta condición. La forma es sencilla pero eficaz y su tamaño pequeño es ideal para empezar en el anal. La ventosa es súper potente, osea que se puede poner en un montón de superficies sin preocuparse de que se esté cayendo continuamente',
 'domingo 12 marzo, 2023',
 '5')

In [133]:
dataframe['user']=dataframe.datos[0]

dataframe['comment']=dataframe.datos[1]
dataframe['date']=dataframe.datos[2]
dataframe['rating']=dataframe.datos[3]

dataframe



ValueError: Length of values (4) does not match length of index (55)